<a href="https://colab.research.google.com/github/KizunaAE/GColabSD/blob/main/KizunaAE-SD-Clone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **StableDiffusion InvokeAI Base Cloud version**



_You don't need additional Google Drive storage because uploaded models are not stored on your Google Drive. After the session ends, all data will be deleted._

In [ ]:
#@markdown # **STEP 1**
#@markdown ## Requirements
#@markdown It might finished with error but is not the error, just execute the next cell

from IPython.display import clear_output

%cd /content
!git clone https://github.com/KizunaAE/InvokeAI
!pip install -q dependency_injector diffusers einops eventlet facexlib flask_cors flask_socketio flaskwebgui getpass_asterisk huggingface-hub
!pip install -q kornia omegaconf pudb pyreadline3 pytorch-lightning realesrgan streamlit taming-transformers-rom1504 test-tube torch-fidelity
!pip install -q torchmetrics transformers picklescan
!pip install -q pillow xformers==0.0.22 triton==2.0.0 -U
clear_output()

!pip install -q git+https://github.com/invoke-ai/GFPGAN@basicsr-1.4.2#egg=gfpgan
!pip install -q git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -q git+https://github.com/Birch-san/k-diffusion.git@mps#egg=k-diffusion
!pip install -q git+https://github.com/invoke-ai/clipseg.git@relaxed-python-requirement#egg=clipseg
!pip install -q git+https://github.com/invoke-ai/PyPatchMatch@0.1.4#egg=pypatchmatch
%cd /content/InvokeAI/
!pip install -q -e .
clear_output()


!wget https://raw.githubusercontent.com/KizunaAE/InvokeAI/main/InvokeAI-colab/main/INITIAL_MODELS.yaml -O /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

!pip install python-socketio==5.9.0

print('[1;32mDone! All models downloaded successfully 🙃')


In [ ]:
#@markdown #STEP 2️⃣: Setup InvokeAI WebUI

#@markdown Keterangan: Wajib untuk Setup Model awal Stable Diffusion. Sekalian bisa pilih Model ControlNET atau abaikan saja.
sd1_controlnet_canny = False # @param {type:"boolean"}
!sed -i "80s/recommended: \(True\|False\)/recommended: $sd1_controlnet_canny/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_inpaint = False # @param {type:"boolean"}
!sed -i "84s/recommended: \(True\|False\)/recommended: $sd1_controlnet_inpaint/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_mlsd = False # @param {type:"boolean"}
!sed -i "88s/recommended: \(True\|False\)/recommended: $sd1_controlnet_mlsd/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_depth = False # @param {type:"boolean"}
!sed -i "92s/recommended: \(True\|False\)/recommended: $sd1_controlnet_depth/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_normal_bae = False # @param {type:"boolean"}
!sed -i "96s/recommended: \(True\|False\)/recommended: $sd1_controlnet_normal_bae/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_seg = False # @param {type:"boolean"}
!sed -i "100s/recommended: \(True\|False\)/recommended: $sd1_controlnet_seg/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_lineart = False # @param {type:"boolean"}
!sed -i "104s/recommended: \(True\|False\)/recommended: $sd1_controlnet_lineart/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_lineart_anime = False # @param {type:"boolean"}
!sed -i "106s/recommended: \(True\|False\)/recommended: $sd1_controlnet_lineart_anime/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_openpose = False # @param {type:"boolean"}
!sed -i "112s/recommended: \(True\|False\)/recommended: $sd1_controlnet_openpose/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_scribble = False # @param {type:"boolean"}
!sed -i "116s/recommended: \(True\|False\)/recommended: $sd1_controlnet_scribble/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_softedge = False # @param {type:"boolean"}
!sed -i "120s/recommended: \(True\|False\)/recommended: $sd1_controlnet_softedge/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_shuffle = False # @param {type:"boolean"}
!sed -i "124s/recommended: \(True\|False\)/recommended: $sd1_controlnet_shuffle/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_tile = False # @param {type:"boolean"}
!sed -i "128s/recommended: \(True\|False\)/recommended: $sd1_controlnet_tile/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

sd1_controlnet_ip2p = False # @param {type:"boolean"}
!sed -i "132s/recommended: \(True\|False\)/recommended: $sd1_controlnet_ip2p/" /content/InvokeAI/invokeai/configs/INITIAL_MODELS.yaml

def Invoke_SetupModels():
    %cd /content/InvokeAI/
    !python /content/InvokeAI/scripts/invokeai-model-install.py --yes --root_dir /content/sd-db

Invoke_SetupModels()

In [ ]:
#@markdown # **STEP 3**
#@markdown ## Run StableDiffusion InvokeAI

import os
import shlex
import subprocess
from pathlib import Path
from typing import Union
clear_output()


id_rsa_file = "/content/InvokeAI/id_rsa"
id_rsa_pub_file = "/content/InvokeAI/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)
clear_output()

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=False)
    path.chmod(0o600)

ssh_name = "id_rsa"
ssh_path = Path(os.path.dirname(os.getcwd())) / ssh_name
gen_key(ssh_path)
clear_output()

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

%cd /content/InvokeAI/
!python /content/InvokeAI/scripts/invokeai-web.py --root /content/db